In [1]:
import cpa
import scanpy as sc

[rank: 0] Global seed set to 0
2024-04-16 08:36:02,082	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-04-16 08:36:09,852	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
from cpa import CPA

In [18]:
import gdown
#gdown.download('https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t')
data_path = 'combo_sciplex_prep_hvg_filtered.h5ad'
adata = sc.read(data_path)

In [19]:
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train.h5ad"
adata_test_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test.h5ad"
adata_ood_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood.h5ad"

In [20]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)


In [21]:
adata_train.X = adata_train.layers["counts"]
adata_test.X = adata_test.layers["counts"]
adata_test.X = adata_test.layers["counts"]

In [22]:

adata.obs["index"] = adata.obs_names.values
adata.obs["index_adapted"] = adata.obs.apply(lambda x: x["index"][:-2], axis=1)
adata.obs.set_index("index_adapted", inplace=True)

In [23]:
adata_train.obs["cov_drug_dose"] = adata.obs['cov_drug_dose']
adata_train.obs["condition_ID"] = adata.obs["condition_ID"]
adata_train.obs["log_dose"] = adata.obs["log_dose"]

adata_test.obs["cov_drug_dose"] = adata.obs['cov_drug_dose']
adata_test.obs["condition_ID"] = adata.obs["condition_ID"]
adata_test.obs["log_dose"] = adata.obs["log_dose"]

adata_ood.obs["cov_drug_dose"] = adata.obs['cov_drug_dose']
adata_ood.obs["condition_ID"] = adata.obs["condition_ID"]
adata_ood.obs["log_dose"] = adata.obs["log_dose"]

In [24]:
frac_valid = adata[adata.obs['split_1ct_MEC']=="valid"].n_obs/(adata[adata.obs['split_1ct_MEC']=="valid"].n_obs+adata[adata.obs['split_1ct_MEC']=="train"].n_obs)

In [25]:
import numpy as np

def create_split(x):
    is_train = np.random.choice(2, p=[frac_valid, 1-frac_valid])
    if is_train:
        return "train"
    return "valid"
adata_train.obs['split_1ct_MEC'] = adata_train.obs.apply(create_split, axis=1)

In [26]:
adata_train.obs['split_1ct_MEC'].value_counts()

split_1ct_MEC
train    46407
valid     5094
Name: count, dtype: int64

In [27]:
adata_train.obs['condition_ID'] = adata_train.obs['condition_ID'].cat.remove_unused_categories()

In [29]:
cpa.CPA.setup_anndata(adata_train,
                      perturbation_key='condition_ID',
                      dosage_key='log_dose',
                      control_group='CHEMBL504',
                      batch_key=None,
                      is_count_data=True,
                      categorical_covariate_keys=['cell_type'],
                      #deg_uns_key='rank_genes_groups_cov',
                      deg_uns_cat_key='cov_drug_dose',
                      max_comb_len=2,
                     )

100%|██████████| 51501/51501 [00:00<00:00, 707552.58it/s]
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [30]:
ae_hparams = {
    "n_latent": 128,
    "recon_loss": "nb",
    "doser_type": "logsigm",
    "n_hidden_encoder": 512,
    "n_layers_encoder": 3,
    "n_hidden_decoder": 512,
    "n_layers_decoder": 3,
    "use_batch_norm_encoder": True,
    "use_layer_norm_encoder": False,
    "use_batch_norm_decoder": True,
    "use_layer_norm_decoder": False,
    "dropout_rate_encoder": 0.1,
    "dropout_rate_decoder": 0.1,
    "variational": False,
    "seed": 434,
}

trainer_params = {
    "n_epochs_kl_warmup": None,
    "n_epochs_pretrain_ae": 30,
    "n_epochs_adv_warmup": 50,
    "n_epochs_mixup_warmup": 3,
    "mixup_alpha": 0.1,
    "adv_steps": 2,
    "n_hidden_adv": 64,
    "n_layers_adv": 2,
    "use_batch_norm_adv": True,
    "use_layer_norm_adv": False,
    "dropout_rate_adv": 0.3,
    "reg_adv": 20.0,
    "pen_adv": 20.0,
    "lr": 0.0003,
    "wd": 4e-07,
    "adv_lr": 0.0003,
    "adv_wd": 4e-07,
    "adv_loss": "cce",
    "doser_lr": 0.0003,
    "doser_wd": 4e-07,
    "do_clip_grad": False,
    "gradient_clip_value": 1.0,
    "step_size_lr": 45,
}

In [31]:
model = cpa.CPA(adata=adata_train,
                split_key='split_1ct_MEC',
                train_split='train',
                valid_split='valid',
                **ae_hparams,
               )

[rank: 0] Global seed set to 434


In [32]:
model.train(max_epochs=2000,
            use_gpu=True,
            batch_size=128,
            plan_kwargs=trainer_params,
            early_stopping_patience=10,
            check_val_every_n_epoch=5,
            save_path='/home/icb/dominik.klein/tmp',
           )

100%|██████████| 27/27 [00:00<00:00, 149.64it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-PCIE-40GB MIG 3g.20gb') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [MIG-0adc0ca7-e2af-58d1-8394-4fe055a61ae9]


Epoch 5/2000:   0%|          | 4/2000 [01:50<13:48:30, 24.91s/it, v_num=1, recon=324, r2_mean=0.758, adv_loss=2.05, acc_pert=0.264]


Epoch 00004: cpa_metric reached. Module best state updated.


Epoch 10/2000:   0%|          | 9/2000 [03:56<13:34:02, 24.53s/it, v_num=1, recon=305, r2_mean=0.77, adv_loss=1.87, acc_pert=0.31, val_recon=317, disnt_basal=0.0674, disnt_after=0.0977, val_r2_mean=0.764, val_KL=nan] 


Epoch 00009: cpa_metric reached. Module best state updated.


...


disnt_basal = 0.051822259641276736
disnt_after = 0.0774848472534904
val_r2_mean = 0.7698026734888554
val_r2_var = 0.15778332856671082
Epoch 25/2000:   1%|          | 24/2000 [10:04<12:27:53, 22.71s/it, v_num=1, recon=282, r2_mean=0.779, adv_loss=1.81, acc_pert=0.343, val_recon=289, disnt_basal=0.0518, disnt_after=0.0775, val_r2_mean=0.77, val_KL=nan]


Epoch 00024: cpa_metric reached. Module best state updated.


Epoch 30/2000:   1%|▏         | 29/2000 [11:54<11:59:20, 21.90s/it, v_num=1, recon=279, r2_mean=0.78, adv_loss=1.83, acc_pert=0.34, val_recon=286, disnt_basal=0.05, disnt_after=0.0765, val_r2_mean=0.776, val_KL=nan]   


Epoch 00029: cpa_metric reached. Module best state updated.



disnt_basal = 0.04868616357164256
disnt_after = 0.0749573109116601
val_r2_mean = 0.7796722971831165
val_r2_var = 0.1595033729432782
Epoch 40/2000:   2%|▏         | 39/2000 [15:38<11:42:27, 21.49s/it, v_num=1, recon=276, r2_mean=0.78, adv_loss=3.38, acc_pert=0.0583, val_recon=284, disnt_basal=0.0376, disnt_after=0.0636, val_r2_mean=0.769, val_KL=nan] 
disnt_basal = 0.03519153009796981
disnt_after = 0.06340509728817394
val_r2_mean = 0.7730453771950124
val_r2_var = 0.14884406946524267
Epoch 50/2000:   2%|▏         | 49/2000 [20:05<15:32:25, 28.68s/it, v_num=1, recon=275, r2_mean=0.783, adv_loss=3.32, acc_pert=0.0476, val_recon=282, disnt_basal=0.0341, disnt_after=0.0647, val_r2_mean=0.775, val_KL=nan]
disnt_basal = 0.034915270692351306
disnt_after = 0.06483346123839044
val_r2_mean = 0.7704133232713934
val_r2_var = 0.14813100807661686
Epoch 60/2000:   3%|▎         | 59/2000 [24:04<11:52:31, 22.03s/it, v_num=1, recon=273, r2_mean=0.783, adv_loss=3.3, acc_pert=0.0454, val_recon=281, disnt_b

In [34]:
cpa.CPA.setup_anndata(adata_test,
                      perturbation_key='condition_ID',
                      dosage_key='log_dose',
                      control_group='CHEMBL504',
                      batch_key=None,
                      is_count_data=True,
                      categorical_covariate_keys=['cell_type'],
                      #deg_uns_key='rank_genes_groups_cov',
                      deg_uns_cat_key='cov_drug_dose',
                      max_comb_len=2,
                     )

100%|██████████| 3100/3100 [00:00<00:00, 698449.85it/s]

INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [36]:
model.predict(adata_test)

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


100%|██████████| 97/97 [00:09<00:00, 10.44it/s]


In [37]:
preds = adata_test.obsm["CPA_pred"]

In [39]:
preds_transformed = np.log1p(preds) - adata_train.varm["X_train_mean"].T

In [40]:
preds_pca = np.matmul(preds_transformed, adata_train.varm["PCs"])

In [44]:
from sklearn.metrics import r2_score
def compute_r_squared(x: np.ndarray, y: np.ndarray) -> float:
    return r2_score(np.mean(x, axis=0), np.mean(y, axis=0))

In [45]:
compute_r_squared(preds_pca, adata_test.obsm["X_pca"])

0.09886690381324148

In [48]:
for cond in adata_test.obs["condition"].unique():
    mask = adata_test.obs["condition"]==cond
    print(compute_r_squared(preds_pca[mask], adata_test.obsm["X_pca"][mask]))

0.34438544073424004
0.1279698933576051
0.3213337024371157
0.10061251122244574
0.36890969872293133
0.13607129870449097
0.08431512902232974
0.25162152604749743
0.5129804403205669
0.5828165329413706
0.2029535479578234
0.18308820882565902
0.5734076802806412
0.5138577994047702
0.18973352856416803
0.14061380055752426
0.4848864092867873
-0.0031861184002539567
0.20230529308478584
0.30049606584391364
0.47576900377466136
0.39210123680807407
0.1010373832260254
0.3210569501636119
0.20962461964054957
0.19873090795045179
0.5496680196167159


In [49]:
model.predict(adata_train)

100%|██████████| 1610/1610 [00:38<00:00, 41.85it/s]


In [50]:
preds_train = adata_train.obsm["CPA_pred"]

In [68]:
preds_transformed = np.log1p(preds_train)# - adata_train.varm["X_train_mean"].T


In [69]:
preds_pca = np.matmul(preds_transformed, adata_train.varm["PCs"])

In [70]:
r_sq = []
for cond in adata_train.obs["condition"].unique():
    mask = adata_train.obs["condition"]==cond
    r_sq.append(compute_r_squared(preds_pca[mask], adata_train.obsm["X_pca"][mask]))

In [71]:
np.mean(r_sq)

0.09509438673943207

In [65]:
for cond in adata_train.obs["condition"].unique():
    mask = adata_train.obs["condition"]==cond
    print(compute_r_squared(preds_train[mask], np.log1p(adata_train.X[mask].A)))

0.7079890896416374
0.7852126407214503
0.7170051077923891
0.7208312405015089
0.7174221488716672
0.7216635689356032
0.7643589130338904
0.7676239033387463
0.7188363671046893
0.7682608480738357
0.767989607435603
0.7178098714201733
0.7212296536460536
0.7236464867354333
0.7716459615276396
0.7748850336795494
0.7700496906585557
0.7584686012892354
0.7317954129796195
0.7847754870348104
0.7634854083538256
0.7110528650012624
0.778188447906329
0.7663576627137513
0.7670877167627943
0.7799535751042828
0.7621037279637786


In [59]:
adata_train.obs["condition"].cat.categories

Index(['Alvespimycin+Pirarubicin', 'Cediranib+PCI-34051',
       'Dacinostat+Dasatinib', 'Dacinostat+PCI-34051', 'Givinostat+Carmofur',
       'Givinostat+Cediranib', 'Givinostat+Crizotinib', 'Givinostat+Curcumin',
       'Givinostat+Dasatinib', 'Givinostat+SRT2104', 'Givinostat+Sorafenib',
       'Givinostat+Tanespimycin', 'Panobinostat+Alvespimycin',
       'Panobinostat+Crizotinib', 'Panobinostat+Dasatinib',
       'Panobinostat+SRT1720', 'Panobinostat+SRT2104', 'Panobinostat+SRT3025',
       'Panobinostat+Sorafenib', 'SRT3025+Cediranib', 'control',
       'control+Alvespimycin', 'control+Dacinostat', 'control+Dasatinib',
       'control+Givinostat', 'control+Panobinostat', 'control+SRT2104'],
      dtype='object')

In [60]:
adata_train[adata_train.obs["condition"]=="control"]

View of AnnData object with n_obs × n_vars = 951 × 2000
    obs: 'sample', 'Size_Factor', 'n.umi', 'RT_well', 'Drug1', 'Drug2', 'Well', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'condition', 'pathway1', 'pathway2', 'split', 'control', 'cell_type', 'perturbation', 'cell_line', 'smiles_drug_1', 'smiles_drug_2', 'cov_drug_dose', 'condition_ID', 'log_dose', 'split_1ct_MEC', 'CPA_cat', 'CPA_CHEMBL504', '_scvi_condition_ID', '_scvi_cell_type', '_scvi_CPA_cat'
    var: 'id', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Drug1_colors', 'Drug2_colors', 'Well_colors', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pathway1_colors', 'pathway2_colors', 'pathway_colors', 'pca', 'rank_genes_groups', 'umap', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: 'X_pca', 'X_umap', 'ecfp_drug_1', 'ecfp_drug_2', 

In [62]:
source = adata_train[adata_train.obs["condition"]=="control"].obsm["X_pca"]
for cond in adata_train.obs["condition"].unique():
    mask = adata_train.obs["condition"]==cond
    print(compute_r_squared(source, adata_train.obsm["X_pca"][mask]))

control+Panobinostat
-4.584203913313214
Cediranib+PCI-34051
0.9687576315115285
Panobinostat+SRT3025
-5.589372510568429
control+Dacinostat
-1.878039579406424
Dacinostat+PCI-34051
-2.804233910226501
Panobinostat+Sorafenib
-5.651873848829433
Givinostat+Cediranib
0.5546720514524565
control+Givinostat
0.581415323480418
Panobinostat+Dasatinib
-5.11655893934242
Givinostat+Tanespimycin
0.21983898888315656
Givinostat+Carmofur
0.44945083012097375
Panobinostat+SRT2104
-4.907405590157562
Panobinostat+SRT1720
-4.276006172490676
Panobinostat+Crizotinib
-6.4672796620208945
Alvespimycin+Pirarubicin
-0.25438013121253866
control
1.0
Givinostat+SRT2104
0.6273838804245343
Givinostat+Sorafenib
-0.3841022121501523
Panobinostat+Alvespimycin
-6.050032366282577
control+Alvespimycin
-0.13728521408259486
Givinostat+Dasatinib
0.5750928475222632
Dacinostat+Dasatinib
-2.578995602988634
control+SRT2104
0.9841166905775888
Givinostat+Curcumin
0.619385224315471
Givinostat+Crizotinib
0.4880410463150848
SRT3025+Cediranib

In [46]:
compute_r_squared(preds_pca[:100], adata_test.obsm["X_pca"][:100])

0.12769485630722444

In [178]:
cpa.CPA.setup_anndata(adata_ood,
                      perturbation_key='condition_ID',
                      dosage_key='log_dose',
                      control_group='CHEMBL504',
                      batch_key=None,
                      is_count_data=True,
                      categorical_covariate_keys=['cell_type'],
                      #deg_uns_key='rank_genes_groups_cov',
                      deg_uns_cat_key='cov_drug_dose',
                      max_comb_len=2,
                     )

100%|██████████| 9277/9277 [00:00<00:00, 917766.78it/s]

INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [179]:
model.predict(adata_test)

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


100%|██████████| 97/97 [00:00<00:00, 127.72it/s]


In [180]:
preds = adata_test.obsm["CPA_pred"]

In [183]:
preds_pca = np.matmul(preds, adata_train.varm["PCs"])

In [192]:
model.predict(adata_train)

100%|██████████| 1610/1610 [00:30<00:00, 51.96it/s]


In [193]:
preds_train = adata_train.obsm["CPA_pred"]

In [199]:
adata_train.X.shape, preds_train.shape

((51501, 2000), (51501, 2000))

In [200]:
compute_r_squared(preds_train, adata_train.X.A)

0.9963734489445446

In [215]:
preds_train_log = np.log1p(preds_train) - adata_train.varm["X_train_mean"].T

In [223]:
(np.log1p(preds_train).mean(axis=0) - adata_train.varm["X_train_mean"]).mean()

-0.015940763

In [221]:
adata_train.varm["X_train_mean"]

array([[0.10696691],
       [0.01764269],
       [0.07884356],
       ...,
       [0.00024302],
       [0.00037494],
       [0.00057457]], dtype=float32)

In [216]:
preds_train_pca = np.matmul(preds_train_log, adata_train.varm["PCs"])

In [ ]:
true_train_pca = np.matmul(adata_train, adata_train.varm["PCs"])

In [218]:
compute_r_squared(preds_train_pca, adata_train.obsm["X_pca"])

-0.01202978864684412

In [219]:
preds_train[0].min(), preds_train[0].max()

(7.5745454e-10, 5.176859)

In [186]:
model.predict(adata_ood)

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


ValueError: Category CHEMBL1213492+CHEMBL257991 not found in source registry. Cannot transfer setup without `extend_categories = True`.

In [ ]:
preds_ood = adata_ood.obsm["CPA_pred"]

In [190]:
"CHEMBL257991" in adata_train.obs["condition_ID"]

False

In [ ]:
"CHEMBL1213492" in adata_train.obs[]

In [138]:
preds_pca = np.matmul(preds, adata_train.varm["PCs"])

In [43]:
from sklearn.metrics import r2_score
def compute_r_squared(x: np.ndarray, y: np.ndarray) -> float:
    return r2_score(np.mean(x, axis=0), np.mean(y, axis=0))

In [153]:
np.isfinite(preds).shape

(3100, 2000)

In [142]:

compute_r_squared(preds_pca, adata_test.obsm["X_pca"])

Epoch 3/2000:   0%|          | 2/2000 [33:26<556:44:02, 1003.12s/it, v_num=1, recon=1.58e+3, r2_mean=0.453, adv_loss=2.55, acc_pert=0.173]


ValueError: Input contains NaN.

In [149]:
(~np.isnan(preds)).sum()

1492000

In [144]:
preds_pca.shape


(3100, 30)

In [145]:
adata_train.varm["PCs"].shape

(2000, 30)

In [156]:
mask = ~np.isnan(preds).any(axis=1)

In [158]:
arr_without_nan = preds[mask]

In [160]:
arr_without_nan.shape

(746, 2000)

In [161]:
preds_pca = np.matmul(arr_without_nan, adata_train.varm["PCs"])

In [184]:
compute_r_squared(preds_pca, adata_test.obsm["X_pca"])

-0.028193542538006877